Welcome to this tutorial! Have you done with the image segmentation  with crowdsourcing and now you need to somehow get the final results? Find out how to aggregate the data with methods from Crowd-Kit library by Toloka!

First of all, let's install and import necessary libraries

In [1]:
%%capture
import pandas as pd
import numpy as np
%pip install crowd-kit

Let's use the mscoco_small data from crowdkit for example and import load_dataset

In [3]:
from crowdkit.datasets import load_dataset
df, gt = load_dataset('mscoco_small')

Unpacking mscoco_small.zip


Let's see the structure of our dataset

In [8]:
df.columns

Index(['task', 'worker', 'segmentation'], dtype='object')

Now let's see how we can solve the aggregation problem with image segmentation. We will use three methods - SegmentationEM, SegmentationMajorityVote, and SegmentationRASA. All these methods solve the same problem - image segmentation aggregation - but in different ways. Let's import them

In [4]:
from crowdkit.aggregation import SegmentationEM
from crowdkit.aggregation import SegmentationMajorityVote
from crowdkit.aggregation import SegmentationRASA

Let's use the EM algorithm for the image segmentation task.

This method performs a categorical aggregation task for each pixel: should it be included to the resulting aggregate or no. This task is solved by the single coin Dawid-Skene algorithm. Each worker has a latent parameter "skill" that shows the probability of this worker to answer correctly. Skills and true pixels' labels are optimized by the Expectation-Maximization algorithm.

In [5]:
result_EM = SegmentationEM().fit_predict(df)

Now, we need to convert our results to numpy array for convenience

In [ ]:
result_EM = result_EM.to_numpy()
gt = gt.to_numpy()

We will use the IOU metric to calculate the efficiency of the methods

In [25]:
IOU_EM = np.array([])
for i in range(0, len(result_EM)-1):
  overlap = result_EM[i]*gt[i] 
  union = result_EM[i] + gt[i] 
  IOU_EM = np.append(IOU_EM, overlap.sum()/float(union.sum()))
IOU_EM.mean()

0.8551840979658429

The next method - Segmentation Majority Vote - chooses a pixel if more than half of workers voted.

This method implements a straightforward approach to the image segmentations aggregation: it assumes that if pixel is not inside in the worker's segmentation, this vote counts as 0, otherwise, as 1. Next, the SegmentationEM aggregates these categorical values for each pixel by the Majority Vote.

In [27]:
result_MajorityVote = SegmentationMajorityVote().fit_predict(df)
result_MajorityVote = result_MajorityVote.to_numpy()

In [33]:
IOU_MajorityVote = np.array([])
for i in range(0, len(result_MajorityVote)-1):
  overlap = result_MajorityVote[i]*gt[i] 
  union = result_MajorityVote[i] + gt[i] 
  IOU_MajorityVote = np.append(IOU_MajorityVote, overlap.sum()/float(union.sum()))
IOU_MajorityVote.mean()

0.8352633625644315

The last method - Segmentation RASA - chooses a pixel if sum of weighted votes of each workers' more than 0.5.

Algorithm works iteratively, at each step, the workers are reweighted in proportion to their distances to the current answer estimation. The distance is considered as 1−IOU. Modification of the RASA method for texts.

In [34]:
result_RASA = SegmentationRASA().fit_predict(df)
result_RASA = result_RASA.to_numpy()

In [36]:
IOU_RASA = np.array([])
for i in range(0, len(result_RASA)-1):
  overlap = result_RASA[i]*gt[i] 
  union = result_RASA[i] + gt[i] 
  IOU_RASA = np.append(IOU_RASA, overlap.sum()/float(union.sum()))
IOU_RASA.mean()

0.8395562415748342

See? All the results are quite good but the SegmentationEM shows slightly better result.
In this tutorial, we looked at how to deal with the image segmentation aggregation task.